# RES Cleaning

This notebook walks through how the RES articles were sorted into categories of articles and non-articles.

## Load Libraries

In [ ]:
from tokenize import Ignore
from numpy import NaN
import pandas as pd
from difflib import SequenceMatcher
from multiprocessing import Pool
import multiprocessing as mp
import time
import pandas as pd
from difflib import SequenceMatcher
from multiprocessing import Pool
import multiprocessing as mp
import time

## Load Files

Replace the file paths below to match local file paths. Note: comment out files that are not available eg: datadumps.

In [53]:
masters = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\Master lists\\RES_master.xlsx")
pivots = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\pivots\\RES_pivots.xlsx")
scopus = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\Scopus\\RES_SCOPUS.xlsx")
datadump = pd.read_excel("C:\\Users\\sjwu1\\Journal_Data\\datadumps\\RES_datadump.xlsx")

pd.set_option('display.max_colwidth', None)

## Create File Names

In [54]:
authors="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\RES_authors.xlsx"
non_auth="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\RES_Nauthors.xlsx"
saveas="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\RES_processed.xlsx"
reviews="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\RES_reviews.xlsx"
misc="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\RES_misc.xlsx"
conf="C:\\Users\\sjwu1\\Journal_Data\\datadumps\\RES_conf.xlsx"

## Some random checks on the masters list

My assumption is that all data without author names must be miscellaneous documents like reports by the committee, forewords, front matters etc.. The goal of this notebook is to check for certain that all the documents without author names are actually miscellaneous documents and then classify them as miscellaneous (MISC). Hence, first we group everything the data by title to see the repetitive general content that can likely be removed.

In [55]:
pd.set_option('display.max_rows',masters.shape[0])
temp=masters['title'].str.lower().value_counts()
pd.DataFrame(temp[temp>1])

,title
front matter,297
back matter,278
volume information,73
accepted manuscripts,9
periodicals received,8
books and monographs received,8
editorial announcement,8
comment,5
editorial,5
introduction,4


Some repetitions are due to multiple comments. Now consider this list in absence of author names.

In [56]:
temp1=masters[masters['authors'].isna()]['title'].str.lower().value_counts()
pd.DataFrame(temp1)

,title
front matter,297
back matter,278
volume information,73
accepted manuscripts,9
books and monographs received,8
periodicals received,8
books and periodicals received,3
announcement,3
introduction,2
errata: neutral inventions and production functions,2


In [57]:
#pd.DataFrame(masters[masters['content_type'].isna()]['title'].str.lower().value_counts())
#masters[masters['title'].str.lower().str.match(r'(^|: )report of the')]
#masters[masters['title'].str.lower().str.match(r'(^|.*: )report of the')]
#masters.loc[masters['title'].str.lower().str.match(r'^combined references(.*)')==True,'content_type']='MISC'

# this is a random panel for testing code

,Unnamed: 0,stable_url,authors,title,abstract,content_type,issue_url,pages


Notice that titles with duplicates of greater or equal to 4 are miscellaneous items and so they are classified in bulk

In [59]:
temp2=masters[(masters['content_type'].isna()==True)]['title'].str.lower().value_counts()
removal=list(temp2[temp2>=4].index)
masters.loc[masters.title.str.lower().isin(removal),'content_type']='MISC'

Classify some titles via regex

In [60]:
masters.loc[masters['title'].str.lower().str.contains('books and periodicals')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^editorial')==True,'content_type']='MISC'
masters.loc[masters['title'].str.lower().str.match(r'^announcement')==True,'content_type']='MISC'
masters.loc[masters['title'].str.strip().str.len()<4,'content_type']='MISC'
#masters[masters.content_type=='MISC']


In [61]:
scopus.rename(columns = {'abstract':'abstract2', 'title':'title2', 'authors':'authors2'}, inplace = True)
scopus['pages2']=scopus['pages']
masters['pages']=masters['pages'].str.strip()  
masters.loc[masters.title.str.lower() == "back matter", 'pages'] = NaN  

## Classifying other content

In [62]:
sum(masters.content_type.isna())
#masters.shape[0]

3259

In [63]:
masters.loc[masters['authors'].str.lower().str.match(r'^review(ed|) by(.*)')==True,'content_type']='Review' #reviews
masters.loc[(masters['title'].str.lower().str.match(r'(.*) by (.*)')==True) & (masters.authors.isna()==True),'content_type']='Review2' 
#possible reviews that don't have author names
masters[(masters['content_type']=='Review2') | (masters['content_type']=='Review')].shape[0] #reviews

0

In [64]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?).*comment.*$')==True,'content_type']='Comment'
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*comment$')==True,'content_type']='Comment'
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'(a further|further) comment.*$')==True,'content_type']='Comment'
masters[masters['content_type']=='Comment'].shape[0] 
#comments

55

In [65]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?) (|a )reply(| to.*)$')==True,'content_type']="Reply"
masters[masters['content_type']=='Reply'].shape[0]

15

In [66]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?|).*rejoinder.*$')==True,'content_type']="Rejoinder"
masters[masters['content_type']=='Rejoinder'].shape[0]


9

In [67]:
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*: (|a )discussion$')==True,'content_type']="Discussion"
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'(^|a )discussion(|.*)$')==True,'content_type']="Discussion"
masters.loc[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*:.*(|a )discussion(|s)$')==True,'content_type']='Discussion'
masters[masters['content_type']=='Discussion'].shape[0]


0

In [68]:
masters.loc[masters['content_type'].isna(),'content_type']="Article"
masters[masters['content_type']=='Article'].shape[0]

3180

In [2]:
# block for testing regex patterns
#masters[masters['title'].str.lower().str.match(r'^\washington notes$')==True]
#masters[masters.content_type.isna() & masters.title.str.lower().str.match(r'.*(:|\?) (|a )rejoinder.*$')==True]
#masters[masters.content_type=='Discussion'].shape[0]

## Consider the pivots file
At times, conference papers are structured differently to normal articles. Hence, it may be necessary to distinguish conference papers from common articles. The next code block separates special issues (S) from normal issues (N)

In [71]:
pivots.loc[pivots.Jstor_issue_text.str.lower().str.match(r'(.*)(supplement|proceedings|annual meeting|survey)(.*)'),'type']="S"
pivots.loc[pivots.type.isna(),'type']='N'
pivots.type.value_counts()

N    309
Name: type, dtype: int64

Merge masterlist and pivot list

In [72]:
result = pd.merge(masters, pivots[['issue_url','year','volume','issue','journal','type']], how="left", on=["issue_url", "issue_url"])

## Summary

In [73]:
pd.DataFrame(result.content_type.value_counts())

,content_type
Article,3180
MISC,711
Comment,55
Reply,15
Rejoinder,9


In [74]:
pd.DataFrame(result[result.year>1939].content_type.value_counts())

,content_type
Article,3036
MISC,670
Comment,54
Reply,11
Rejoinder,7


In [75]:
pd.DataFrame(result[(result.year>1939) & (result.year<2011)].content_type.value_counts())

,content_type
Article,2608
MISC,606
Comment,54
Reply,11
Rejoinder,7


In [76]:
result.to_excel(saveas, index=False)